<a href="https://colab.research.google.com/github/Suryan5h/Natural-Language-Processing/blob/main/basics/Gesture_Recognition__Assignment_SuryanshBhardwaj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
#from scipy.misc import imread, imresize
import cv2
import imageio
import datetime
import os
#from PIL import Image, ImageFilter, ImageEnhance

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_doc = np.random.permutation(open('/content/drive/MyDrive/Colab Notebooks/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/drive/MyDrive/Colab Notebooks/Project_data/val.csv').readlines())
batch_size = 32

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
#using percentile to deal with outliers in the data.
def normalize_image(img):
    #using percentile for normalization for images, as min-max is giving better results.
    # normalized_image= img - np.percentile(img,15)/ np.percentile(img,85) - np.percentile(img,15) 
    normalized_image= (img - np.min(img))/(np.max(img)- np.min(img))
    return normalized_image

In [6]:
seq_idx = range(7,26,3)

In [25]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = seq_idx
    x = len(img_idx)
    # image dimensions
    y, z = 120, 120
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if image.shape[1] > image.shape[0]:
                        diff_px = image.shape[1] - image.shape[0]
                        crop_start = diff_px//2
                        crop_end = crop_start + image.shape[0]
                        image = image[:, crop_start:crop_end]
                    elif image.shape[0] > image.shape[1]:
                        diff_px = image.shape[0] - image.shape[1]
                        crop_start = diff_px//2
                        crop_end = crop_start + image.shape[1]
                        image = image[:, crop_start:crop_end]

                    resized_im = cv2.resize(image, dsize=(y,z))
                    
                    batch_data[folder,idx,:,:,0] = normalize_image(resized_im[:, : , 0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = normalize_image(resized_im[:, : , 1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = normalize_image(resized_im[:, : , 2])#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if((len(source_path)%batch_size)//2==0):
            batch_size = 2
        else:
            batch_size = 1
        num_batches = len(source_path)%batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    
                    
                    # Cropped image of above dimension 
                    # (It will not change orginal image) 
                    
                    #image = image.crop((0, 0, 120, 120))
                    image = image.resize(100, 100)
                    
                    if image.shape[1] > image.shape[0]:
                        diff_px = image.shape[1] - image.shape[0]
                        crop_start = diff_px//2
                        crop_end = crop_start + image.shape[0]
                        image = image[:, crop_start:crop_end]
                    elif image.shape[0] > image.shape[1]:
                        diff_px = image.shape[0] - image.shape[1]
                        crop_start = diff_px//2
                        crop_end = crop_start + image.shape[1]
                        image = image[:, crop_start:crop_end]

                    resized_im = cv2.resize(image, dsize=(y,z))
                    
                    batch_data[folder,idx,:,:,0] = normalize_image(resized_im[:, : , 0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = normalize_image(resized_im[:, : , 1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = normalize_image(resized_im[:, : , 2])#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [26]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/drive/MyDrive/Colab Notebooks/Project_data/train'
val_path = '/content/drive/MyDrive/Colab Notebooks/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [27]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Rescaling, Dropout, GlobalAveragePooling3D, ConvLSTM2D
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
#input shape for the st layer
input_shape = (len(seq_idx), 120, 120, 3)

Experiment - 1 (Conv3D)

In [10]:
# model = Sequential()
# model.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
# model.add(MaxPooling3D(pool_size=1))

# model.add(Conv3D(64, kernel_size=3, activation='relu'))
# model.add(MaxPooling3D(pool_size=2))

# model.add(Flatten())
# model.add(Dense(16, activation='relu'))
# model.add(Dense(5, activation='softmax'))

Experiment - 2 (Conv3D - As Model is underfit, need more training)

In [11]:
# model = Sequential()

# model.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape,padding='same'))
# model.add(Conv3D(64, kernel_size=3, activation='relu', padding='same'))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# model.add(Conv3D(128, kernel_size=3, activation='relu',padding='same'))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu',padding='same'))

# model.add(Conv3D(512, kernel_size=(3, 3, 3), activation='relu',padding='same'))
# model.add(Conv3D(512, kernel_size=(3, 3, 3), activation='relu',padding='same'))
# model.add(MaxPooling3D(pool_size=1))

# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dense(5, activation='softmax'))

Experiment - 3

In [12]:
# model = Sequential()

# model.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape,padding='same'))
# model.add(Conv3D(64, kernel_size=3, activation='relu',padding='same'))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# model.add(Conv3D(128, kernel_size=3, activation='relu',padding='same'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2)))

# model.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu',padding='same'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2)))

# model.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu',padding='same'))
# model.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu',padding='same'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2)))

# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dense(5, activation='softmax'))

Experiment - 4 (Batch Normalization and adding dropout layers in between)

In [13]:
# model = Sequential()

# model.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape,padding='same'))
# model.add(Conv3D(64, kernel_size=3, activation='relu',padding='same'))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

# model.add(Conv3D(128, kernel_size=3, activation='relu',padding='same'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2)))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

# model.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu',padding='same'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2)))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

# model.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu',padding='same'))
# model.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu',padding='same'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2)))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dense(5, activation='softmax'))

Model 4 results:
1. Epoch 27, Training accuracy = 99.55%
2. Training Loss = 0.03778
3. Validation Accuracy = 64.02% (Highest till Model 4)
4. Validation Loss = 1.36040

So, this model is performing good with high training accuracy and moderate validation accuracy after doing batch normalization and adding dropout layers for solving overfitting.

Experiment - 5

In [14]:
# model = Sequential()

# model.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
# model.add(Conv3D(64, kernel_size=3, activation='relu',padding='same'))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# model.add(Conv3D(128, kernel_size=3, activation='relu',padding='same'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2)))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# model.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu',padding='same'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2)))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# model.add(GlobalAveragePooling3D())
# model.add(Dense(512, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dense(5, activation='softmax'))

Model 5 results:
1. Epoch 50, Training accuracy = 89.28%
2. Training Loss = 0.28310
3. Validation Accuracy = 71.875% (Highest till Model 5)
4. Validation Loss = 0.9690

So, this model is performing really well with moderate training accuracy (Neither overfitting nor underfitting) and moderate validation accuracy after experimenting with Conv3D layers, Dropout layer and using GlobalAveragePooling3D.
<br>
This is a pretty good model using Conv3D. Now we will try to experiment with TimeDistributed.

Experiment - 6 (Time Distributed Conv2D+ GRU)

In [15]:
# Importing Libraries
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D

In [16]:
# model = Sequential()
# model.add(TimeDistributed(
#     Conv2D(32, (3,3), activation='relu'), input_shape=input_shape)
# )
# model.add(TimeDistributed(
#     MaxPooling2D((2,2)))
# )
# model.add(BatchNormalization())

# model.add(TimeDistributed(
#     Conv2D(64, (3,3), activation='relu',padding='same'))
# )
# model.add(TimeDistributed(
#     MaxPooling2D((2,2)))
# )
# model.add(BatchNormalization())

# model.add(TimeDistributed(GlobalAveragePooling2D()))
# model.add(TimeDistributed(Dense(64, activation='relu')))
# model.add(BatchNormalization())

# model.add(GRU(128))
# model.add(BatchNormalization())
# model.add(Dense(5, activation='softmax'))

Model 6 results:

Epoch 45, Training accuracy = 88.988% <br>
Training Loss = 0.31143 <br>
Validation Accuracy = 75% (Highest till Model 6) <br>
Validation Loss = 0.96900.57760 <br>
So, this model is performing really well without any tuning on training accuracy (Neither overfitting nor underfitting) and good validation accuracy (75%) using TimeDistributed Conv2D+ GRU architecture.

Experiment - 7 (Added dropout layers in model 6)

In [17]:
model = Sequential()
model.add(TimeDistributed(
    Conv2D(32, (3,3), activation='relu'), input_shape=input_shape)
)
model.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(TimeDistributed(
    Conv2D(64, (3,3), activation='relu',padding='same'))
)
model.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(TimeDistributed(GlobalAveragePooling2D()))
model.add(TimeDistributed(Dense(64, activation='relu')))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(GRU(128))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))

Model 7 results:

Epoch 45, Training accuracy = 93.155% <br>
Training Loss = 0.20165 <br>
Validation Accuracy = 78.90% (Highest till Model 7) <br>
Validation Loss = 0.63995 <br>
So, this model is performing really well after adding dropout layers with impressive validation accuracy (78.9%) using TimeDistributed Conv2D+ GRU architecture.

Experiment - 8 (Time Distributed Conv2D+ ConvLSTM2D)<br>
###NOTE: Commented out Experiment 8 instead of experiment 7 because Experiment 7 is our final model with the best accuracy. But the last model fitted was experiment 8 (implemented below) which is less accurate than Model/Experiment 7.

In [18]:
# model = Sequential()
# model.add(TimeDistributed(
#     Conv2D(8, (3,3), activation='relu'), input_shape=input_shape)
# )
# model.add(BatchNormalization())
# model.add(TimeDistributed(
#     Conv2D(16, (3,3), activation='relu'))
# )
# model.add(BatchNormalization())
# model.add(
#     ConvLSTM2D(8, kernel_size = 3, return_sequences=False)
# )
# model.add(BatchNormalization())
# model.add(TimeDistributed(
#     Dense(64, activation='relu'))
# )
# model.add(BatchNormalization())
# model.add(GlobalAveragePooling2D())
# model.add(Dense(64, activation='relu'))
# model.add(Dense(5, activation='softmax'))

In [19]:
optimiser = optimizers.Adam(lr=0.01) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 7, 118, 118, 8)   224       
 ibuted)                                                         
                                                                 
 batch_normalization (BatchN  (None, 7, 118, 118, 8)   32        
 ormalization)                                                   
                                                                 
 time_distributed_1 (TimeDis  (None, 7, 116, 116, 16)  1168      
 tributed)                                                       
                                                                 
 batch_normalization_1 (Batc  (None, 7, 116, 116, 16)  64        
 hNormalization)                                                 
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 114, 114, 8)       6

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [28]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [21]:
model_name = 'model_8_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model8-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [22]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [23]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-23-bd77c9c60c14>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  /content/drive/MyDrive/Colab Notebooks/Project_data/train ; batch size = 32
Epoch 1/50
21/21 [==============================] - ETA: 0s - loss: 1.4475 - categorical_accuracy: 0.3304  Source path =  /content/drive/MyDrive/Colab Notebooks/Project_data/val ; batch size = 32

Epoch 1: saving model to model_8_init_2023-03-0815_13_03.329141/model8-00001-1.44746-0.33036-1.74603-0.23438.h5
21/21 [==============================] - 1399s 69s/step - loss: 1.4475 - categorical_accuracy: 0.3304 - val_loss: 1.7460 - val_categorical_accuracy: 0.2344 - lr: 0.0100
Epoch 2/50
21/21 [==============================] - ETA: 0s - loss: 1.2452 - categorical_accuracy: 0.4301
Epoch 2: saving model to model_8_init_2023-03-0815_13_03.329141/model8-00002-1.24523-0.43006-1.99802-0.14062.h5
21/21 [==============================] - 75s 4s/step - loss: 1.2452 - categorical_accuracy: 0.4301 - val_loss: 1.9980 - val_categorical_accuracy: 0.1406 - lr: 0.0100
Epoch 3/50
21/21 [=============================

Model 8 results:

Epoch 50, Training accuracy = 81.25% <br>
Training Loss = 0.4871 <br>
Validation Accuracy = 74.22% <br>
Validation Loss = 0.7819 <br>
So, this model is performing fine but not better than Experiment 7 (Time Distributed Conv2D+ GRU architecture)

## CONCLUSION

Model 7 is our best model.
Model 7 results:

Epoch 45, Training accuracy = 93.155% <br>
Training Loss = 0.20165 <br>
Validation Accuracy = 78.90% (BEST AND FINAL MODEL) <br>
Validation Loss = 0.63995 <br>
So, this model is performing really well after adding dropout layers with impressive validation accuracy (78.9%) using TimeDistributed Conv2D+ GRU architecture.

#### The End